In [13]:
import pandas as pd
import geopandas as gpd
import numpy as np
import geopy
import altair as alt
import openml
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
import scikit_posthocs as sp

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import plotly.graph_objects as go

from tqdm import tqdm
import time

alt.data_transformers.enable("vegafusion")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('vegafusion')

In [14]:
# Alterar isso

dataset = openml.datasets.get_dataset(42080)
df, _, _, _ = dataset.get_data(dataset_format='dataframe')

In [19]:
df = pd.read_csv("amostra.csv", index_col=0)
df

,cmte_id,amndt_ind,rpt_tp,transaction_pgi,transaction_tp,entity_tp,city,state,zip_code,employer,occupation,transaction_dt,transaction_amt,file_num
1079858,C00034157,N,M5,Desconhecido,15,IND,MINNEAPOLIS,MN,55439.0,missing,retired,2012-04-01,6.030685,783390.0
1346510,C00505552,A,30G,G,15E,IND,BETHESDA,MD,20816.0,missing,retired,2012-11-04,5.521461,857344.0
8888,C00328211,N,Q1,P,15,IND,CHEVY CHASE,MD,20815.0,missing,retired,2011-03-15,6.907755,726710.0
2502508,C00506931,A,12C,C,15,IND,ARLINGTON,VA,22203.0,missing,retired,2012-01-25,6.907755,850871.0
1128381,C00476564,N,Q2,P,15,IND,WASHINGTON,DC,20003.0,missing,retired,2012-06-21,5.521461,798378.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070271,C00431171,A,M3,P,15,IND,VALDOSTA,GA,31601.0,missing,retired,2012-02-24,6.907755,780124.0
222486,C00495028,N,YE,Desconhecido,10,PAC,WASHINGTON,DC,20005.0,missing,retired,2011-12-20,8.517193,763491.0
3042627,C00431304,A,30G,G,15,IND,CLAYTON,MO,63105.0,missing,retired,2012-10-27,6.214608,878471.0
2884697,C00498634,N,12G,G,15,IND,MUNDELEIN,IL,60060.0,missing,retired,2012-10-10,6.214608,827719.0


# Data Visualization

In [20]:
colorscale = ['#0b132b', '#1b2740', '#2a3c56',
              '#354964', '#3a506b', '#3a506b',
              '#57b1b1', '#63d8cd', ' #6fffe9']

## 1. Perfis mais comums de Contribuintes

In [21]:
columns = ['amndt_ind', 'rpt_tp', 'transaction_pgi', 'transaction_tp', 'entity_tp', 'occupation']
freq_table = df.groupby(columns).size().reset_index(name='counts')
freq_table = freq_table.sort_values('counts', ascending=False)

data = freq_table.head(9)
sankey = data.copy()

In [22]:
sankey

,amndt_ind,rpt_tp,transaction_pgi,transaction_tp,entity_tp,occupation,counts
291,A,Q3,P,15,IND,retired,1968
261,A,Q2,P,15,IND,retired,1756
14,A,12G,G,15,IND,retired,1342
317,A,YE,P,15,IND,retired,1227
54,A,30G,G,15,IND,retired,1128
85,A,M10,G,15,IND,retired,1008
238,A,Q1,P,15,IND,retired,937
212,A,M9,P,15,IND,retired,918
577,N,Q2,P,15,IND,retired,889


In [23]:
#@title .

sankey = sankey[['transaction_tp', 'entity_tp', 'amndt_ind',
                 'occupation', 'transaction_pgi', 'rpt_tp', 'counts']]

original_attributes = ['transaction_tp', 'entity_tp', 'amndt_ind',
                       'occupation', 'transaction_pgi', 'rpt_tp']

display_attributes = ['Tipo de Transação', 'Tipo de Entidade', 'Indicador de Emenda',
                      'Ocupação', 'Etapa do Ciclo Eleitoral', 'Tipo de Relatório']

attribute_name_mapping = dict(zip(original_attributes, display_attributes))


node_labels = []
node_colors = []
node_attrs = []
node_indices = {}
color_palette = ["#eff7f6", "#a3abb2", "#707885", "#3e4658", "#242c42", "#0b132b",
                 "#2a3c56", "#354964", "#3a506b", "#57b1b1", "#63d8cd", "#6fffe9"]


attr_colors = {attr: color_palette[i % len(color_palette)] for i, attr in enumerate(original_attributes)}

current_index = 0
for attr in original_attributes:
    values = sankey[attr].unique()
    for val in values:
        node_name = str(val)
        if node_name not in node_indices:
            node_indices[node_name] = current_index
            node_labels.append(node_name)
            node_colors.append(attr_colors[attr])

            node_attrs.append(attribute_name_mapping[attr])
            current_index += 1


links = []
node_in_counts = {index: 0 for index in range(len(node_labels))}
node_out_counts = {index: 0 for index in range(len(node_labels))}

for _, row in sankey.iterrows():
    counts = row['counts']
    for i in range(len(original_attributes) - 1):
        source_attr = original_attributes[i]
        target_attr = original_attributes[i + 1]
        source_node = str(row[source_attr])
        target_node = str(row[target_attr])

        source_index = node_indices[source_node]
        target_index = node_indices[target_node]


        base_color = node_colors[source_index]
        link_color = base_color.replace('#', '0x')
        rgb_color = tuple(int(link_color[i:i+2], 16) for i in (2, 4, 6))
        rgba_color = f'rgba({rgb_color[0]}, {rgb_color[1]}, {rgb_color[2]}, 0.4)'

        links.append({
            'source': source_index,
            'target': target_index,
            'value': counts,
            'color': rgba_color
        })

        node_out_counts[source_index] += counts
        node_in_counts[target_index] += counts

node_counts = {}
for node_index in range(len(node_labels)):
    node_counts[node_index] = max(node_in_counts[node_index], node_out_counts[node_index])

sankey_nodes = {
    'label': node_labels,
    'color': node_colors,
    'customdata': list(zip(
        node_attrs,
        [node_counts[index] for index in range(len(node_labels))]
    )),
    'hovertemplate': '%{customdata[0]}: %{label}<br>Número de contribuintes: %{customdata[1]}<extra></extra>'
}

sankey_links = {
    'source': [link['source'] for link in links],
    'target': [link['target'] for link in links],
    'value': [link['value'] for link in links],
    'color': [link['color'] for link in links],
    'hovertemplate': (
        'Origem: %{source.label}<br>' +
        'Destino: %{target.label}<br>' +
        'Número de contribuintes: %{value}<extra></extra>'
    )
}

fig = go.Figure(go.Sankey(
    arrangement='snap',
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=sankey_nodes['label'],
        color=sankey_nodes['color'],
        customdata=sankey_nodes['customdata'],
        hovertemplate=sankey_nodes['hovertemplate']
    ),
    link=dict(
        source=sankey_links['source'],
        target=sankey_links['target'],
        value=sankey_links['value'],
        color=sankey_links['color'],
        hovertemplate=sankey_links['hovertemplate']
    )
))

# Título do gráfico
fig.update_layout(
    title_text="Perfis mais comuns dos contribuintes",
    font_size=12,
    paper_bgcolor="rgba(0,0,0,0)",
    font_color='black'
)

# Exibir o Sankey Diagram
fig.show()


In [ ]:
fig.write_html("sankey_diagram.html")

## 2. Proporção dos tipos de entidades, Indicadores de Emenda e tipos de transações.

In [24]:
encoding = df['amndt_ind'].value_counts().reset_index().copy()
total = encoding['count'].sum()
encoding['percentage'] = encoding['count']/total
encoding

,amndt_ind,count,percentage
0,A,22198,0.665049
1,N,11162,0.334412
2,T,18,0.000539


In [ ]:
chart.save('setores.html')

In [25]:
encoding = df['entity_tp'].value_counts().reset_index().copy()
total = encoding['count'].sum()
encoding['percentage'] = encoding['count']/total
encoding

,entity_tp,count,percentage
0,IND,33139,0.992840
1,ORG,90,0.002696
2,CAN,81,0.002427
3,PAC,42,0.001258
4,CCM,16,0.000479
5,COM,9,0.000270
6,PTY,1,0.000030


In [26]:
encoding = df['transaction_tp'].value_counts().reset_index().copy()
total = encoding['count'].sum()
encoding['percentage'] = encoding['count']/total
encoding

,transaction_tp,count,percentage
0,15,29967,0.897807
1,24T,1065,0.031907
2,15E,883,0.026455
3,22Y,781,0.023399
4,10,330,0.009887
5,24I,214,0.006411
6,15C,103,0.003086
7,11,30,0.000899
8,19,3,0.000090
9,20Y,2,0.000060


## 3. Quais etapas do Ciclo Eleitoral recebem mais contribuições

In [27]:
# Data enconding for vis
quantidade_contribuicoes = df['transaction_pgi'].value_counts().reset_index()
quantidade_contribuicoes.columns = ['transaction_pgi', 'quantidade_contribuicoes']

quantia_total = df.groupby('transaction_pgi')['transaction_amt'].sum().reset_index()
quantia_total.columns = ['transaction_pgi', 'quantia_total']

vis1 = pd.merge(quantidade_contribuicoes, quantia_total, on='transaction_pgi')
mapeamento = {
    'P': 'Primária (Interna)',
    'G': 'Geral',
    'S': 'Especial',
    'E': 'Eleição',
    'C': 'Convenção',
    'O': 'Outras',
    'R': 'Segundo Turno',
    'Desconhecido': 'Desconhecido',
    '0': '0'
}

vis1['transaction_pgi'] = vis1['transaction_pgi'].map(mapeamento)

vis1 = vis1.sort_values(by='quantidade_contribuicoes', ascending=False)
vis1['razao_contribuicao'] = vis1['quantia_total']/vis1['quantidade_contribuicoes']

vis1

,transaction_pgi,quantidade_contribuicoes,quantia_total,razao_contribuicao
0,Primária (Interna),15723,98751.561515,6.280707
1,Desconhecido,9313,56853.624557,6.104759
2,Geral,7963,48710.107761,6.117055
3,Convenção,133,836.565705,6.289968
4,Outras,100,681.808193,6.818082
5,Segundo Turno,69,452.077913,6.551854
6,Especial,68,436.082308,6.412975
7,Eleição,5,34.985063,6.997013
8,0,4,27.854165,6.963541


In [28]:

# Criação da seleção interativa para o highlight ao passar o mouse
highlight = alt.selection_point(on='mouseover', fields=['transaction_pgi'], empty='none')

# Escala de cor personalizada
color_scale = alt.Scale(
    domain=vis1['transaction_pgi'].unique(),
    range=colorscale
)

# Gráfico de barras com highlight ao passar o mouse
bar = alt.Chart(vis1).mark_bar().encode(
    x=alt.X('transaction_pgi', title='Etapa do Ciclo Eleitoral', sort='-y'),
    y=alt.Y('quantia_total:Q', title='Quantia Total de Contribuições (em USD)'),
    color=alt.condition(
        highlight,
        alt.Color('transaction_pgi:N', scale=color_scale, title='Etapa'),
        alt.value('lightgray')  # Cor padrão para barras não destacadas
    ),
    tooltip=[
        alt.Tooltip('transaction_pgi:N', title='Etapa do Ciclo'),
        alt.Tooltip('quantia_total:Q', title='Quantia total de contribuição'),
        alt.Tooltip('razao_contribuicao:Q', title='Proporção Quantia/Contribuintes')
    ]

).add_params(
    highlight
).properties(
    width=600,
    height=300,
    title='Quantia Total de Contribuições por Etapa do Ciclo Eleitoral'
)

# Texto com valor, sem alterações
text = alt.Chart(vis1).mark_text(
    align='center',
    baseline='middle',
    dy=-10,
    color='black'
).encode(
    x=alt.X('transaction_pgi:N', title='Etapa do Ciclo Eleitoral', sort='-y'),
    y=alt.Y('quantia_total:Q'),
    text=alt.Text('razao_contribuicao:Q', format='.2f')
)

# Combinação do gráfico de barras e texto com configuração de estilo
vis = (bar + text).configure_title(
    fontSize=18,
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
).configure_legend(
    labelFontSize=12,
    titleFontSize=14,
).configure(
    background="rgba(0, 0, 0, 0)"
).configure_view(
    fill='white'
).configure_axis(
    grid=False,
)

vis.display()


alt.LayerChart(...)

In [ ]:
vis.save('vis1.html')

In [29]:
#@title 3.2
color_scale = alt.Scale(
    domain=vis1['transaction_pgi'].unique()[3:],
    range=colorscale[3:]
)

bar = alt.Chart(vis1).transform_filter(
    alt.FieldOneOfPredicate(field='transaction_pgi', oneOf=['Outras', 'Convenção', 'Especial', 'Segundo Turno', 'Eleição', '0'])
).mark_bar().encode(
    x=alt.X('transaction_pgi', title='Etapa do Ciclo Eleitoral', sort='-y'),
    y=alt.Y('quantia_total:Q', title='Quantia Total de Contribuições (em USD)'),
    tooltip=[
        alt.Tooltip('transaction_pgi:N', title='Etapa do Ciclo'),
        alt.Tooltip('quantia_total:Q', title='Quantia total de contribuição'),
        alt.Tooltip('razao_contribuicao:Q', title='Proporção Quantia/Contribuintes')],
    color=alt.Color('transaction_pgi:N', scale=color_scale, title='Etapa'),
).properties(
    width=600,
    height=300,
    title='Quantia Total de Contribuições por Etapa do Ciclo Eleitoral (sem classes majoritárias)'
)

text = alt.Chart(vis1).transform_filter(
    alt.FieldOneOfPredicate(field='transaction_pgi', oneOf=['Outras', 'Convenção', 'Especial', 'Segundo Turno', 'Eleição', '0'])
).mark_text(
    align='center',
    baseline='middle',
    dy=-10,
    color='black'
).encode(
    x=alt.X('transaction_pgi:N', title='Etapa do Ciclo Eleitoral', sort='-y'),
    y=alt.Y('quantia_total:Q'),
    text=alt.Text('razao_contribuicao:Q', format='.2f')
)

vis = (bar + text).configure_title(
    fontSize=18
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_legend(
    labelFontSize=12,
    titleFontSize=14
).configure(
    background="rgba(0, 0, 0, 0)"
).configure_view(
    fill='white'
).configure_axis(
    grid=False
)
vis.display()

alt.LayerChart(...)

In [ ]:
vis.save('vis2.html')

In [30]:
# Escala de cor personalizada para as categorias selecionadas
color_scale = alt.Scale(
    domain=vis1['transaction_pgi'].unique()[3:],
    range=colorscale[3:]
)

# Seleção interativa para o destaque ao passar o mouse
highlight = alt.selection_point(on='mouseover', fields=['transaction_pgi'], empty='none')

# Gráfico de barras com filtro e efeito de highlight ao passar o mouse
bar = alt.Chart(vis1).transform_filter(
    alt.FieldOneOfPredicate(field='transaction_pgi', oneOf=['Outras', 'Convenção', 'Especial', 'Segundo Turno', 'Eleição', '0'])
).mark_bar().encode(
    x=alt.X('transaction_pgi', title='Etapa do Ciclo Eleitoral', sort='-y'),
    y=alt.Y('quantia_total:Q', title='Quantia Total de Contribuições (em USD)'),
    tooltip=[
        alt.Tooltip('transaction_pgi:N', title='Etapa do Ciclo'),
        alt.Tooltip('quantia_total:Q', title='Quantia total de contribuição'),
        alt.Tooltip('razao_contribuicao:Q', title='Proporção Quantia/Contribuintes')
    ],
    color=alt.condition(
        highlight,
        alt.Color('transaction_pgi:N', scale=color_scale, title='Etapa'),
        alt.value('lightgray')  # Cor padrão para barras não destacadas
    )
).add_params(
    highlight
).properties(
    width=600,
    height=300,
    title='Quantia Total de Contribuições por Etapa do Ciclo Eleitoral (sem classes majoritárias)'
)

# Texto com valores numéricos
text = alt.Chart(vis1).transform_filter(
    alt.FieldOneOfPredicate(field='transaction_pgi', oneOf=['Outras', 'Convenção', 'Especial', 'Segundo Turno', 'Eleição', '0'])
).mark_text(
    align='center',
    baseline='middle',
    dy=-10,
    color='black'
).encode(
    x=alt.X('transaction_pgi:N', title='Etapa do Ciclo Eleitoral', sort='-y'),
    y=alt.Y('quantia_total:Q'),
    text=alt.Text('razao_contribuicao:Q', format='.2f')
)

# Combinação das camadas e configurações de estilo
vis = (bar + text).configure_title(
    fontSize=18
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_legend(
    labelFontSize=12,
    titleFontSize=14
).configure(
    background="rgba(0, 0, 0, 0)"
).configure_view(
    fill='white'
).configure_axis(
    grid=False
)

vis.display()


alt.LayerChart(...)

In [ ]:
vis.save('vis3.html')

### 3.4 Existem diferenças entre as distribuições apresentadas?

In [ ]:
df_analysis = df[['transaction_pgi', 'transaction_amt']].copy()

groups = [group['transaction_amt'].values for name, group in df_analysis.groupby('transaction_pgi')]

stat, p = kruskal(*groups)
print(f'Estatística H de Kruskal-Wallis: {stat:.4f}')
print(f'p-valor: {p:.4e}')

Estatística H de Kruskal-Wallis: 38497.3239
p-valor: 0.0000e+00


In [ ]:
post_hoc = sp.posthoc_dunn(df_analysis, val_col='transaction_amt', group_col='transaction_pgi', p_adjust='fdr_bh')
pares_significantes = (post_hoc < 0.05)
significancia_dos_estados = pares_significantes.sum(axis=1)
significancia_dos_estados.sort_values(ascending=False)

,0
0,8
Desconhecido,8
G,8
O,8
P,8
C,7
E,7
R,7
S,7


Para comparar as distribuições acima, utiliza-se o teste estatístico **Kruskal-Wallis** com o intuito de verificar se existem diferenças significativas entre as medianas desses grupos. Sob a Hipótese nula (H0) de que as medianas dos grupos são iguais, obtem-se o p-valor < 0.05 e, assim, H0 é rejeitado.

Assim, para entender onde está a diferença entre esses grupos, faz-se o teste post-hoc de Dunn. Ele realiza comparações múltiplas entre pares dos grupos, aplicando testes Mann-Whitney U para cada par, ajustando os p-valores para múltiplas comparações.

Apesar dos resultados mostrarem que existem diferenças entre as distribuições das etapas do ciclo Eleitoral, procura-se entender o efeito prático dessa diferença: verificar se a diferença entre as distribuições é relevante nesse contexto. Para isso, calcula-se medidas de Tamanho de efeito.

In [ ]:
n_total = len(df_analysis)
k = len(groups)
epsilon_squared = (stat - k + 1) / (n_total - k)
print(f'Epsilon quadrado: {epsilon_squared:.6f}')

Epsilon quadrado: 0.011531


O Epsilon quadrado representa a proporção da variabilidade total dos dados que é explicada pela variável independente, nesse caso, as etapas de ciclo eleitoral. Seu intervalo varia de 0 a 1 e pelo seu cálculo, percebe-se que o tamanho do efeito das diferenças das distribuições é insignificante.

In [ ]:
def calculate_effect_size(group1, group2):
    n1 = len(group1)
    n2 = len(group2)
    u_statistic, _ = mannwhitneyu(group1, group2, alternative='two-sided')
    u_max = n1 * n2
    effect_size = 1 - (2 * u_statistic) / u_max
    return abs(effect_size)

group_dict = {
    name: group['transaction_amt'].values
    for name, group in df_analysis.groupby('transaction_pgi')
}

group_names = list(group_dict.keys())
n_groups = len(group_names)

effect_size_array = np.zeros((n_groups, n_groups))

for i in range(n_groups):
    group1 = group_names[i]
    data1 = group_dict[group1]

    for j in range(i, n_groups):
        group2 = group_names[j]
        data2 = group_dict[group2]
        effect_size = calculate_effect_size(data1, data2)
        effect_size_array[i, j] = effect_size
        effect_size_array[j, i] = effect_size

# Converter a matriz NumPy em DataFrame
effect_size_matrix = pd.DataFrame(
    effect_size_array,
    index=group_names,
    columns=group_names
)


In [ ]:
#@title .
effect_size_long = effect_size_matrix.reset_index().melt(
    id_vars='index',
    var_name='Grupo 2',
    value_name='Tamanho de Efeito'
)
effect_size_long.rename(columns={'index': 'Grupo 1'}, inplace=True)

# Criar o heatmap
heatmap = alt.Chart(effect_size_long).mark_rect().encode(
    x=alt.X('Grupo 2:N', title='Etapa do Ciclo Eleitoral '),
    y=alt.Y('Grupo 1:N', title='Etapa do Ciclo Eleitoral'),
    color=alt.Color('Tamanho de Efeito:Q', scale=alt.Scale(scheme='lightgreyteal')),
    tooltip=['Grupo 1', 'Grupo 2', alt.Tooltip('Tamanho de Efeito:Q', format='.4f')]
).properties(
    width=400,
    height=350,
    title='Matriz de Tamanho de Efeito entre Etapas do Ciclo Eleitoral'
)

text= alt.Chart(effect_size_long).mark_text(baseline='middle').encode(
    x='Grupo 2:O',
    y='Grupo 1:O',
    text=alt.Text('Tamanho de Efeito:Q', format='.2f'),
)

vis = (heatmap + text).configure_title(
    fontSize=18,

).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_legend(
    labelFontSize=12,
    titleFontSize=14
).configure(
    background="rgba(0, 0, 0, 0)"
).configure_view(
    fill='white'
).configure_axis(
    labelAngle=-45
)

# Exibir o heatmap
vis.display()

alt.LayerChart(...)

In [ ]:
#@title .
# Remover a coluna '0' e a linha '0' da matriz de tamanhos de efeito
effect_size_matrix_filtered = effect_size_matrix.drop(index='0', columns='0')

# Transformar a matriz filtrada em formato longo
effect_size_long_filtered = effect_size_matrix_filtered.reset_index().melt(
    id_vars='index',
    var_name='Grupo 2',
    value_name='Tamanho de Efeito'
)
effect_size_long_filtered.rename(columns={'index': 'Grupo 1'}, inplace=True)

import altair as alt

# Criar o heatmap com a matriz filtrada
heatmap = alt.Chart(effect_size_long_filtered).mark_rect().encode(
    x=alt.X('Grupo 2:N', title='Etapa do Ciclo Eleitoral'),
    y=alt.Y('Grupo 1:N', title='Etapa do Ciclo Eleitoral'),
    color=alt.Color('Tamanho de Efeito:Q', scale=alt.Scale(scheme='lightgreyteal')),
    tooltip=['Grupo 1', 'Grupo 2', alt.Tooltip('Tamanho de Efeito:Q', format='.4f')]
).properties(
    width=400,
    height=350,
    title='Matriz de Tamanho de Efeito entre Etapas do Ciclo Eleitoral'
)

# Adicionar os valores de tamanho de efeito como texto no heatmap
text = alt.Chart(effect_size_long_filtered).mark_text(baseline='middle').encode(
    x='Grupo 2:N',
    y='Grupo 1:N',
    text=alt.Text('Tamanho de Efeito:Q', format='.2f'),
)

# Combinar o heatmap e o texto
vis = (heatmap + text).configure_title(
    fontSize=18,
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    labelAngle=-45
).configure_legend(
    labelFontSize=12,
    titleFontSize=14
).configure(
    background="rgba(0, 0, 0, 0)"
).configure_view(
    fill='white'

).configure_axis(
    labelAngle=-45
)

# Exibir o heatmap
vis.display()


alt.LayerChart(...)

# Further Explorations

Para a segunda entrega, espera-se explorar como as variáveis estão distribuídas no espaço e no tempo. Busca-se comprender a autocorrelação espacial e encontrar padrões na Série.

Além disso, explora-se técnicas de redução de dimensionalidade considerando as muitas variáveis categóricas. A partir das embeddings das categorias pode ser possível criar representações semânticas para as classes, possivelmente criando um espaço de dimensão pequena no qual haja agrupamentos dos contribuintes.



# References

Thomas, Tressy, and Enayat Rajabi. "A systematic review of machine learning-based missing value imputation techniques." Data Technologies and Applications 55.4 (2021): 558-585.

https://www.fec.gov/campaign-finance-data/
committee-type-code-descriptions/  
https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/
